In [50]:
from typing import Dict
import warnings
import pandas as pd
import gspread as gs 
import calendar as cd
import streamlit as st
import plotly.graph_objects as go
warnings.simplefilter(action='ignore', category=pd.core.common.SettingWithCopyWarning)


def fetch_data():
    print("""
    # --------------------------------- #
    # Connection to Google Sheet        #
    # and extract Spreadsheets          #
    #  - exported tricount data         #
    #  - category dictonnary            #
    # --------------------------------- #
    """)
    gc = gs.service_account_from_dict(st.secrets['gcp_service_account'])
    ss = gc.open_by_key(st.secrets['tricount'].spreadsheet_key)
    data = pd.DataFrame(ss.worksheet('Data').get_all_records())
    dict = pd.DataFrame(ss.worksheet('Dict').get_all_records())
    data.to_csv('.out/data.csv')
    dict.to_csv('.out/dict.csv')
    data = pd.read_csv('.out/data.csv')
    dict = pd.read_csv('.out/dict.csv')
    dict = dict[['Postes', 'Catégories']].set_index('Catégories').to_dict()['Postes']
    print(" -- data fetched")
    print(data.shape)
    print(data.columns)
    print(" -- dict fetched ")
    print(dict)
    return data, dict


def build_data(data: pd.DataFrame):
    print("""
    # --------------------------------- #
    # Filter, clean and aggregate data  #
    # --------------------------------- #
    """)
    data = data[data['Catégorie'] != ""]
    data['Date'] = pd.to_datetime(data['Date & heure'], format='%d/%m/%Y %H:%M')
    data['Année'] = data['Date'].dt.year
    data['Mois'] = data['Date'].dt.month
    data['Jour'] = data['Date'].dt.day
    data['Lucie'] = data['Impacté à Lucie']
    data['Vincent'] = data['Impacté à Vincent']

    data = data.groupby(['Année', 'Mois', 'Catégorie']).agg({'Lucie': "sum", 'Vincent': "sum"})
    data['Total'] = data['Lucie'] + data['Vincent']

    print(" -- data built")
    print(data.shape)
    print(data.columns)
    return data[['Total', 'Lucie', 'Vincent']]


def split_data(data: pd.DataFrame, dict: Dict):
    print("""
    # --------------------------------- #
    # Format and split data             #
    # by date and categories            #
    # --------------------------------- #
    """)
    detail: Dict[str, pd.DataFrame] = {}
    years = data.index.get_level_values('Année').unique().to_list()
    for year in sorted(years, reverse=True):
        df = data.loc[year,:,:]
        months = df.index.get_level_values('Mois').unique().tolist()
        detail[f'{year} (sum)'] = df.groupby('Catégorie').sum()
        detail[f'{year} (mean)'] = detail[f'{year} (sum)'] / len(months)

        for month in sorted(months, reverse=True):
            period = f'{year} {cd.month_name[month]}'
            detail[period] = df.loc[month,:].groupby('Catégorie').sum()

    for period in detail.keys():
        # print(period)
        for category in set(dict.keys()):
            if category not in detail[period].index:
                # print(f"       --> add category: {category} to period {period}")
                detail[period].loc[category] = [0, 0, 0]

    return detail



In [52]:
data, dict = fetch_data()
data = build_data(data)
detail = split_data(data, dict)
# postes, result = concat_data(detail, dict)


    # --------------------------------- #
    # Connection to Google Sheet        #
    # and extract Spreadsheets          #
    #  - exported tricount data         #
    #  - category dictonnary            #
    # --------------------------------- #
    
 -- data fetched
(1023, 17)
Index(['Unnamed: 0', 'Titre', 'Montant', 'Devise', 'Taux de change',
       'Montant dans la devise du tricount (EUR)', 'Type de transaction',
       'Catégorie', 'Payé par', 'Payé par Commun', 'Payé par Lucie',
       'Payé par Vincent', 'Impacté à Commun', 'Impacté à Lucie',
       'Impacté à Vincent', 'Date & heure', 'URL des images'],
      dtype='object')
 -- dict fetched 
{'Alimentation': 'Quotidien', 'Shopping': 'Achats', 'Logement': 'Achats', 'Loyer & charges': 'Quotidien', 'Divertissement': 'Extra', 'Sport': 'Loisir', 'Restaurant & bar': 'Extra', 'Investissement': 'Investissement', 'Transport': 'Quotidien', 'Week-end': 'Loisir', 'Soin de santé': 'Quotidien', 'Livres': 'Achats', 'Activités': 'Lois

In [58]:
from typing import Dict
import warnings
import pandas as pd
import gspread as gs 
import calendar as cd
import streamlit as st
import plotly.graph_objects as go
warnings.simplefilter(action='ignore', category=pd.core.common.SettingWithCopyWarning)
def concat_data(detail: Dict[str, pd.DataFrame], dict: pd.DataFrame):
    print("""
    # --------------------------------- #
    # Build the overview result         #
    # --------------------------------- #
    """)
    postes: Dict[str, pd.DataFrame] = {}
    result: Dict[str, pd.DataFrame] = {}
    for period in detail.keys():
        dfp = detail[period].reset_index()
        dfp['Poste'] = dfp['Catégorie'].apply(lambda category: dict[category])
        dfp = dfp.set_index('Poste')[['Total', 'Lucie', 'Vincent']]
        dfp = dfp.groupby('Poste').sum()
        postes[period] = dfp.copy()
        # for poste in set(dict.values()):
        #     if poste not in df.columns:
        #         df.loc[poste] = [0, 0, 0]

        dfr = pd.DataFrame()
        dfp = dfp.transpose()
        dfr['Revenus'] = dfp['Rentrée d\'argent']
        dfr['Dépenses'] = dfp['Quotidien'] + dfp['Loisir'] + dfp['Extra'] + dfp['Achats']
        dfr['Reste à vivre'] = dfr['Revenus'] + dfr['Dépenses']
        dfr['Reste à vivre %'] = dfr['Reste à vivre'] / dfr['Revenus'] * 100
        dfr['Capital investi'] = - dfp['Investissement']
        dfr['Capital investi %'] = dfr['Capital investi'] / dfr['Revenus'] * 100
        dfr['Epargne'] = dfr['Reste à vivre'] - dfr['Capital investi']
        dfr['Epargne %'] = dfr['Epargne'] / dfr['Revenus'] * 100
        result[period] = dfr.transpose()

    return postes, result

postes, result = concat_data(detail, dict)
result


    # --------------------------------- #
    # Build the overview result         #
    # --------------------------------- #
    


{'2023 (sum)':                     Total    Lucie  Vincent
 Revenus               0.0     0.00     0.00
 Dépenses          -1061.0 -1019.65   -41.35
 Reste à vivre     -1061.0 -1019.65   -41.35
 Reste à vivre %      -inf     -inf     -inf
 Capital investi    -106.0    -0.00  -106.00
 Capital investi %    -inf      NaN     -inf
 Epargne            -955.0 -1019.65    64.65
 Epargne %            -inf     -inf      inf,
 '2023 (mean)':                     Total    Lucie  Vincent
 Revenus               0.0     0.00     0.00
 Dépenses          -1061.0 -1019.65   -41.35
 Reste à vivre     -1061.0 -1019.65   -41.35
 Reste à vivre %      -inf     -inf     -inf
 Capital investi    -106.0    -0.00  -106.00
 Capital investi %    -inf      NaN     -inf
 Epargne            -955.0 -1019.65    64.65
 Epargne %            -inf     -inf      inf,
 '2023 January':                     Total    Lucie  Vincent
 Revenus               0.0     0.00     0.00
 Dépenses          -1061.0 -1019.65   -41.35
 Reste à